<center><img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="30%" /></center>

<hr style="height:2px;border:none"/>
<h1 align='center'> ELI-349 Operación de Sistemas Eléctricos de Potencia </h1>

<h2 align='center'><b>Tarea Final</b></h2>
<h3 align='center'>Preparado por Miguel Huerta</h3>



<hr style="height:2px;border:none"/>


In [5]:
import os, getpass
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from platform import python_version
from scipy.sparse import csr_matrix as sparse
from time import time


np.set_printoptions(edgeitems=30, linewidth=100000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.rcParams.update({
        'lines.linewidth' : 2,
        'legend.fontsize' : 16,
        'font.size'       : 16,
        'axes.titlesize'  : 18, 
        'axes.labelsize'  : 18,
        'xtick.labelsize' : 16,
        'ytick.labelsize' : 16,
        'figure.dpi'      : 80,
        'figure.facecolor': 'w',
        'figure.edgecolor': 'k',
        })


########### Digsilent
try: 
  cwd = os.getcwd()
  Digsilent = os.listdir("C:\Program Files\DIgSILENT\\")
  try:    DigsilentVersion = Digsilent[['2020' in k for k in Digsilent].index(True)]
  except: DigsilentVersion = Digsilent[['2019' in k for k in Digsilent].index(True)]
  os.chdir("C:\Program Files\DIgSILENT\\"+DigsilentVersion+'\\Python\\'+python_version()[:3])
  import powerfactory as pf
  try:    app = pf.GetApplication(getpass.getuser(),'') 
  except: app = pf.GetApplication(User,'') 
  os.chdir(cwd)
  app.ResetCalculation()
  print(DigsilentVersion)
except Exception as e: print(e)

########### Gurobi
try: 
  cwd = os.getcwd()
  ls  = os.listdir("C:\\")
  for gu in ls: 
    if 'gurobi' in gu: 
      os.chdir("C:\\"+gu+'\win64\python'+python_version()[:-2].replace('.','')+'\lib\gurobipy')
      import gurobipy as gp
      from gurobipy import GRB
      print('Gurobi')
  os.chdir(cwd)
except:
  pass

PowerFactory 2020 SP1
Gurobi


## Funciones Digsilent

In [22]:
def searchDB(user, name=None):
  NoElements = ['RecBin', 'Set', 'Study Cases','Network Model',
  'Operation Scenarios','Settings','Library','CIM Model', 'Step Response Test', 'Test commands'
  ]
  for projectName in user.GetChildren(1):
    if projectName.loc_name.strip() not in NoElements:
      print(projectName.loc_name)
      if name==projectName.loc_name:
        proj = projectName
      else:
        proj = None
      try:
        for projectName2 in projectName.GetChildren(1):
          if projectName2.loc_name.strip() not in NoElements:
            print('   ', projectName2.loc_name)
            if name==projectName2.loc_name:
              proj = projectName2
            else:
              proj = None
      except:pass
  return proj

def DefineOuputChannels(elmRes):
  buses, syms, loads, stats = [app.GetCalcRelevantObjects("*."+k) for k in ['ElmTerm', 'ElmSym', 'ElmLod', 'ElmGenStat']]
  for k in elmRes.GetChildren(1): k.Delete() # reset
  for bus in buses:  elmRes.AddVars(bus, 'm:u', 'm:phiu','m:fehz',)   
  for sym in syms:   elmRes.AddVars(sym, 's:ve','s:pt','s:ut','s:ie','s:xspeed','s:xme','s:xmt','s:cur1','s:P1','s:Q1','c:firel',)
  for load in loads: elmRes.AddVars(load, 'n:u1:bus1', 'm:I1:bus1', 'm:Psum:bus1', 'm:Qsum:bus1')
  for stat in stats: elmRes.AddVars(stat, 'm:Psum:bus1', 'm:Qsum:bus1')
  return elmRes

def clearSimEvents():
  faultFolder = app.GetFromStudyCase("Simulation Events/Fault.IntEvt")
  cont = faultFolder.GetContents()
  for obj in cont:
    obj.Delete()

def ExportResults(comres, elmRes, name='temp'):
  comres.ciopt_head = 1
  comres.iopt_csel  = 0
  comres.iopt_tsel  = 0
  comres.iopt_locn  = 1
  comres.iopt_sep   = 1
  comres.pResult    = elmRes
  comres.f_name     = os.getcwd()+'\\'+name+'.csv'
  comres.Execute()

def runSimulation(comInc, comSim, app):
  app.EchoOff()
  comInc.Execute()
  app.EchoOn()
  comSim.Execute()

def setupSimultation(comInc, comSim):
  comInc.iopt_sim = "rms"
  comInc.iopt_show = 0
  comInc.iopt_adapt = 0
  comInc.dtgrd = 0.01
  comInc.start = -0.1
  comSim.tstop = 15

In [57]:
user = app.GetCurrentUser()
searchDB(user)

respaldo
    BD 2026 V4.3
    BD ERST 2020-2026v1.2
    BD ERST 2020-2026v1.4POC
    BD ERST 2020-2026v1.5
    BD ERST 2020-2026v1.5POC
    BD ERST 2020 14-09v1.6PH
    BD ERST 2020-2026v1.7POC-9-23-1
    14B-Tunning
    14B-Tunning(1)
    14 Bus System
BD ERST 2020-2026v2.1POC(7-8-9)
BD ERST 2020-2026v2.0
FAPI_B
FAPI_A
Droop
VSP
IEEE 14-bus AGCv1.3PVSystem
    AGC simulation
PPT2
    2 Two Area System with WADC
    2 Two Area System without WADC
2 Two Area System with WADC
2 Two Area System without WADC


In [54]:
version = '2 Two Area System without WADC'

try:
  project = app.ActivateProject(version) 
  prj = app.GetActiveProject()
  print('Nombre del Proyecto: {:s}'.format(prj.loc_name))
  comLdf = app.GetFromStudyCase("ComLdf")    # Load flow
  comInc = app.GetFromStudyCase('ComInc')    # Dynamic initialization
  comSim = app.GetFromStudyCase('ComSim')    # Transient simulations
  comMod = app.GetFromStudyCase('ComMod')    # Modal Analysis
  comRes = app.GetFromStudyCase('ComRes')
  events = app.GetFromStudyCase('IntEvt')
  comMod.iopt_met = 0 # QR/QZ-Method
  EigenValues = comMod.ResultFile
  


  
  comInc.Execute()
  comMod.Execute()
  try:os.remove(os.getcwd()+'\\temp.csv')
  except: pass

except Exception as e: print(e)



Nombre del Proyecto: 2 Two Area System without WADC


In [128]:
#bus	Pg	Qg	Qmax	Qmin	Vg	mBase	status	Pmax	Pmin	Pc1	Pc2	Qc1min	Qc1max	Qc2min	Qc2max	ramp_agc	ramp_10	ramp_30	ramp_q	apf 

syms  = {k.loc_name:k for k in app.GetCalcRelevantObjects("*.ElmSym")}

for k in syms:
  print(k,
        syms[k].ip_ctrl,  # Ref
        syms[k].bus1.cterm.loc_name
        )

G01 0 1
G02 0 1
G03 1 1
G04 0 1


In [95]:
#bus_i	type	Pd	Qd	Gs	Bs	area	Vm	Va	baseKV	zone	Vmax	Vmin
buses = {k.loc_name:k for k in app.GetCalcRelevantObjects("*.ElmTerm")}


In [110]:
loads = {k.loc_name:k for k in app.GetCalcRelevantObjects("*.ElmLod")}
for k in loads:
    print(
        k,
        loads[k].plini,
        loads[k].qlini,
    )

L 07 967.0 100.0
L 09 1767.0 100.0


In [127]:
#fbus	tbus	r	x	b	rateA	rateB	rateC	ratio	angle	status	angmin	angmax
lines = {k.loc_name:k for k in app.GetCalcRelevantObjects("*.ElmLne")}
for k in lines:
    print(
        k,
        str(lines[k].bus1.cterm).split('\\')[-2],
        # lines[k].bus2.loc_name,
        # lines[k].R1,
        # lines[k].X1
    )

L 05-06 B05.ElmSubstat
L 06-07 B06.ElmSubstat
L 07-08-1 B07.ElmSubstat
L 07-08-2 B07.ElmSubstat
L 08-09-1 B08.ElmSubstat
L 08-09-2 B08.ElmSubstat
L 09-10 B09.ElmSubstat
L 10-11 B11.ElmSubstat


In [138]:
Substat  = {k.loc_name:k for k in app.GetCalcRelevantObjects("*.ElmSubstat")}

for k in Substat:
  print(k,)
  for k1 in Substat[k].GetChildren(1):
      print(str(k1))


B05
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\B05.ElmSubstat\CB1.ElmCoup</l3>
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\B05.ElmSubstat\CB2.ElmCoup</l3>
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\B05.ElmSubstat\CB3.ElmCoup</l3>
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\B05.ElmSubstat\CB4.ElmCoup</l3>
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\B05.ElmSubstat\IS1.1.ElmCoup</l3>
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\B05.ElmSubstat\IS1.2.ElmCoup</l3>
<l3>\Miguel.IntUser\2 Two Area System without WADC.IntPrj\Netw